In [ ]:
import h5py
import pandas as pd
import time, datetime, calendar
import numpy as np
import panel as pn
import datetime as dt
import holoviews as hv
from holoviews.operation.datashader import rasterize
from skyfield.api import EarthSatellite
hv.extension('bokeh')

In [ ]:
tle = pd.read_csv('./data/tle2017.csv')

In [ ]:
'Total number of TLEs in tle2017.csv: %s' %  len(tle)

In [ ]:
'Total number of unique satellite IDs: %s' % len(tle['SatNum'].unique())

In [ ]:
tle.head()

In [ ]:
easting, northing = hv.util.transform.lon_lat_to_easting_northing(tle['Lon'], tle['Lat'])

In [ ]:
(hv.element.tiles.ESRI().redim(x='longitude', y='latitude') 
 * rasterize(hv.Points((easting, northing))).redim(x='longitude', y='latitude').opts(width=700, height=600))

## Visualizing export_test.h5 TLEs

In [ ]:
f = h5py.File('../vault/export_test.h5', 'r')
dataset = f['readout']

In [ ]:
'TLE records in export_test.h5: %s' % len(dataset)

In [ ]:
print('Example TLE record:\n%s' % dataset[0])

In [ ]:
def compute_TLE_easting_northing(n=1000000):
    coords = []
    for ind in range(n):
        line1 = bytes(dataset[ind][2]).decode('ASCII').replace('\\', '')
        line2 = bytes(dataset[ind][3]).decode('ASCII').replace('\\', '')
        satellite = EarthSatellite(line1, line2, name='Unknown')
        geometry = satellite.at(satellite.epoch)

        subpoint = geometry.subpoint()
        if np.isnan(subpoint.latitude.degrees):
            continue
        if np.isnan(subpoint.longitude.degrees):
            continue
        
        easting, northing = hv.util.transform.lon_lat_to_easting_northing(subpoint.longitude.degrees, 
                                                                          subpoint.latitude.degrees)
        coords.append((easting, northing))
    return coords

In [ ]:
coords = compute_TLE_easting_northing(n=1000000) # Takes a while to run for n=1000000

In [ ]:
hv.element.tiles.ESRI() * rasterize(hv.Points(coords)).opts(width=700, height=400)